In [2]:
# File Directory 
import glob
import os
from os.path import isdir, join
from pathlib import Path

# Math
import numpy as np
from scipy.fftpack import fft
from scipy import signal
import librosa

# Dimension Reduction
from sklearn.decomposition import PCA

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as ipd
import librosa.display
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls

# Data Pre-processing
import pandas as pd
from sklearn.model_selection import KFold
#import soundfile

# Deep Learning
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import Input, layers
from tensorflow.keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

# Configuration
py.init_notebook_mode(connected=True)

#%matplotlib inline

In [19]:
# EDITED TO HAVE ONLY 2 LABELS: 1 for gunshot, 0 for everything else

samples=[]
sample_rates=[]
labels = []
sample_slice_iteration = 0
gunshot_aggregator = {}
glassbreak_aggregator = {}

#gunshot_sound_dir = "/Users/laurenogden/school/iupui/gunshot/data/gunshot/"
gunshot_sound_dir = "/home/lauogden/data/gunshot/"

for file in os.listdir(gunshot_sound_dir):
    if file.endswith(".wav"):
        try:
            #print("FILENAME: " + gunshot_sound_dir + file)
            sample, sample_rate = librosa.load(gunshot_sound_dir + file)
            #print("    read the file " + file)

            #add the 2 second long samples to the list of sample slices
            if len(sample) < 44100:
                sample_slice = np.zeros((44100,))
                sample_slice = sample[0 : sample.size]
                samples.append(sample_slice)
                sample_rates.append(sample_rate)
                label = 1
                gunshot_aggregator[sample_slice_iteration] = np.max(abs(sample_slice))
                sample_slice_iteration += 1
                if np.max(abs(sample_slice)) < 0.25:
                    label = 0
                labels.append(label)
                #print("    appended the slice")
                #print("    size of samples is: " + str(len(samples)))

            #slice clips longer than 2 seconds long
            for i in range(0, sample.size - 44100, 44100):
                sample_slice = sample[i : i + 44100]
                #print("    sliced the file " + file)
                label = 1
                gunshot_aggregator[sample_slice_iteration] = np.max(abs(sample_slice))
                sample_slice_iteration += 1
                if np.max(abs(sample_slice)) < 0.25:
                    label = 0

                samples.append(sample_slice)
                sample_rates.append(sample_rate)
                labels.append(label)
                #print("    appended the slice")
                #print("    size of samples is: " + str(len(samples)))

        except:
            sample, sample_rate = soundfile.read(gunshot_sound_dir + file)
            print("Gunshot sound unrecognized by Librosa:", sample)
            pass
        
print("size of samples currently = " + len(samples))
#glassbreak_sound_dir = "/Users/laurenogden/school/iupui/gunshot/data/glassbreak/"
glassbreak_sound_dir = "/home/lauogden/data/glassbreak/"

print("...Switching to glassbreak sounds...")

for file in os.listdir(glassbreak_sound_dir):
    if file.endswith(".wav"):
        try:
            sample, sample_rate = librosa.load(glassbreak_sound_dir + file)
            print("    read the file" + file)
            for i in range(0, sample.size - 44100, 44100):
                sample_slice = sample[i : i + 44100]
                print("    sliced the file" + file)
                label = 0
                glassbreak_aggregator[sample_slice_iteration] = np.max(abs(sample_slice))
                sample_slice_iteration += 1
                # if np.max(abs(sample_slice)) < 0.5:
                    # label = 0

                samples.append(sample_slice)
                sample_rates.append(sample_rate)
                labels.append(label)
                #print("    appended the slice")
        except:
            sample, sample_rate = soundfile.read(glassbreak_sound_dir + file)
            print("Glassbreak sound unrecognized by Librosa:", sample)
            pass

print("size of samples is now: " + len(samples))

KeyboardInterrupt: 

In [ ]:
#read in the csv file of descriptors for all other urban sounds
#sound_types = pd.read_csv("/Users/laurenogden/school/iupui/gunshot/data/train/train.csv")
sound_types = pd.read_csv("/home/lauogden/data/train/train.csv")
print(sound_types.loc[0,'Class'])



urban_aggregator = {}
j=0
#read in all of the wav files similar to above
#urban_sound_dir = "/Users/laurenogden/school/iupui/gunshot/data/train/Train/"
urban_sound_dir = "/home/lauogden/data/train/Train/"

print(os.listdir(urban_sound_dir))

for file in os.listdir(urban_sound_dir):
    if file.endswith(".wav"):
        try:
            sample, sample_rate = librosa.load(urban_sound_dir + file)
            print("    read the file" + file)
            for i in range(0, sample.size - 44100, 44100):
                sample_slice = sample[i : i + 44100]
                print("    sliced the file" + file)
                if(sound_types.loc[j, 'Class'] == "gun_shot"):
                    label = 1
                else:
                    label = 0
                urban_aggregator[sample_slice_iteration] = np.max(abs(sample_slice))
                sample_slice_iteration += 1
                if np.max(abs(sample_slice)) < 0.25:
                    label = 0

                samples.append(sample_slice)
                sample_rates.append(sample_rate)
                labels.append(label)
                print("    appended the slice")
            j +=1
        except:
            sample, sample_rate = soundfile.read(urban_sound_dir + file)
            print("Urban sound unrecognized by Librosa:", sample)
            pass


siren
['4666.wav', '2217.wav', '7409.wav', '1078.wav', '6717.wav', '2571.wav', '4100.wav', '6703.wav', '8048.wav', '4672.wav', '1722.wav', '8060.wav', '3135.wav', '5544.wav', '3653.wav', '8706.wav', '1044.wav', '8712.wav', '5236.wav', '3647.wav', '4128.wav', '3121.wav', '4896.wav', '6059.wav', '1736.wav', '8074.wav', '7347.wav', '249.wav', '6918.wav', '3860.wav', '5587.wav', '275.wav', '1087.wav', '513.wav', '3848.wav', '507.wav', '7384.wav', '261.wav', '2773.wav', '6515.wav', '8538.wav', '4464.wav', '2001.wav', '7179.wav', '1508.wav', '922.wav', '4316.wav', '6529.wav', '1246.wav', '2997.wav', '3451.wav', '5746.wav', '2029.wav', '3337.wav', '4458.wav', '7151.wav', '8262.wav', '1520.wav', '7145.wav', '1534.wav', '3323.wav', '2983.wav', '3445.wav', '5034.wav', '1252.wav', '7623.wav', '8510.wav', '7810.wav', '8289.wav', '739.wav', '7804.wav', '2954.wav', '3492.wav', '5949.wav', '5791.wav', '6298.wav', '7838.wav', '705.wav', '2940.wav', '3486.wav', '2607.wav', '3519.wav', '6461.wav', '1468

librosa read the file ok #0
librosa read the file ok #1
librosa read the file ok #2
librosa read the file ok #3
librosa read the file ok #4
librosa read the file ok #5
librosa read the file ok #6
librosa read the file ok #7
librosa read the file ok #8
librosa read the file ok #9
librosa read the file ok #10
librosa read the file ok #11
librosa read the file ok #12
librosa read the file ok #13
librosa read the file ok #14
librosa read the file ok #15
librosa read the file ok #16
librosa read the file ok #17
librosa read the file ok #18
librosa read the file ok #19
librosa read the file ok #20
librosa read the file ok #21
librosa read the file ok #22
librosa read the file ok #23
librosa read the file ok #24
librosa read the file ok #25
librosa read the file ok #26
librosa read the file ok #27
librosa read the file ok #28
librosa read the file ok #29
librosa read the file ok #30
librosa read the file ok #31
librosa read the file ok #32
librosa read the file ok #33
librosa read the file ok

librosa read the file ok #278
librosa read the file ok #279
librosa read the file ok #280
librosa read the file ok #281
librosa read the file ok #282
librosa read the file ok #283
librosa read the file ok #284
librosa read the file ok #285
librosa read the file ok #286
librosa read the file ok #287
librosa read the file ok #288
librosa read the file ok #289
librosa read the file ok #290
librosa read the file ok #291
librosa read the file ok #292
librosa read the file ok #293
librosa read the file ok #294
librosa read the file ok #295
librosa read the file ok #296
librosa read the file ok #297
librosa read the file ok #298
librosa read the file ok #299
librosa read the file ok #300
librosa read the file ok #301
librosa read the file ok #302
librosa read the file ok #303
librosa read the file ok #304
librosa read the file ok #305
librosa read the file ok #306
librosa read the file ok #307
librosa read the file ok #308
librosa read the file ok #309
librosa read the file ok #310
librosa re

librosa read the file ok #553
librosa read the file ok #554
librosa read the file ok #555
librosa read the file ok #556
librosa read the file ok #557
librosa read the file ok #558
librosa read the file ok #559
librosa read the file ok #560
librosa read the file ok #561
librosa read the file ok #562
librosa read the file ok #563
librosa read the file ok #564
librosa read the file ok #565
librosa read the file ok #566
librosa read the file ok #567
librosa read the file ok #568
librosa read the file ok #569
librosa read the file ok #570
librosa read the file ok #571
librosa read the file ok #572
librosa read the file ok #573
librosa read the file ok #574
librosa read the file ok #575
librosa read the file ok #576
librosa read the file ok #577
librosa read the file ok #578
librosa read the file ok #579
librosa read the file ok #580
librosa read the file ok #581
librosa read the file ok #582
librosa read the file ok #583
librosa read the file ok #584
librosa read the file ok #585
librosa re

librosa read the file ok #827
librosa read the file ok #828
librosa read the file ok #829
librosa read the file ok #830
librosa read the file ok #831
librosa read the file ok #832
librosa read the file ok #833
librosa read the file ok #834
librosa read the file ok #835
librosa read the file ok #836
librosa read the file ok #837
librosa read the file ok #838
librosa read the file ok #839
librosa read the file ok #840
librosa read the file ok #841
librosa read the file ok #842
librosa read the file ok #843
librosa read the file ok #844
librosa read the file ok #845
librosa read the file ok #846
librosa read the file ok #847
librosa read the file ok #848
librosa read the file ok #849
librosa read the file ok #850
librosa read the file ok #851
librosa read the file ok #852
librosa read the file ok #853
librosa read the file ok #854
librosa read the file ok #855
librosa read the file ok #856
librosa read the file ok #857
librosa read the file ok #858
librosa read the file ok #859
librosa re

librosa read the file ok #1097
librosa read the file ok #1098
librosa read the file ok #1099
librosa read the file ok #1100
librosa read the file ok #1101
librosa read the file ok #1102
librosa read the file ok #1103
librosa read the file ok #1104
librosa read the file ok #1105
librosa read the file ok #1106
librosa read the file ok #1107
librosa read the file ok #1108
librosa read the file ok #1109
librosa read the file ok #1110
librosa read the file ok #1111
librosa read the file ok #1112
librosa read the file ok #1113
librosa read the file ok #1114
librosa read the file ok #1115
librosa read the file ok #1116
librosa read the file ok #1117
librosa read the file ok #1118
librosa read the file ok #1119
librosa read the file ok #1120
librosa read the file ok #1121
librosa read the file ok #1122
librosa read the file ok #1123
librosa read the file ok #1124
librosa read the file ok #1125
librosa read the file ok #1126
librosa read the file ok #1127
librosa read the file ok #1128
librosa 

librosa read the file ok #1363
librosa read the file ok #1364
librosa read the file ok #1365
librosa read the file ok #1366
librosa read the file ok #1367
librosa read the file ok #1368
librosa read the file ok #1369
librosa read the file ok #1370
librosa read the file ok #1371
librosa read the file ok #1372
librosa read the file ok #1373
librosa read the file ok #1374
librosa read the file ok #1375
librosa read the file ok #1376
librosa read the file ok #1377
librosa read the file ok #1378
librosa read the file ok #1379
librosa read the file ok #1380
librosa read the file ok #1381
librosa read the file ok #1382
librosa read the file ok #1383
librosa read the file ok #1384
librosa read the file ok #1385
librosa read the file ok #1386
librosa read the file ok #1387
librosa read the file ok #1388
librosa read the file ok #1389
librosa read the file ok #1390
librosa read the file ok #1391
librosa read the file ok #1392
librosa read the file ok #1393
librosa read the file ok #1394
librosa 

librosa read the file ok #1629
librosa read the file ok #1630
librosa read the file ok #1631
librosa read the file ok #1632
librosa read the file ok #1633
librosa read the file ok #1634
librosa read the file ok #1635
librosa read the file ok #1636
librosa read the file ok #1637
librosa read the file ok #1638
librosa read the file ok #1639
librosa read the file ok #1640
librosa read the file ok #1641
librosa read the file ok #1642
librosa read the file ok #1643
librosa read the file ok #1644
librosa read the file ok #1645
librosa read the file ok #1646
librosa read the file ok #1647
librosa read the file ok #1648
librosa read the file ok #1649
librosa read the file ok #1650
librosa read the file ok #1651
librosa read the file ok #1652
librosa read the file ok #1653
librosa read the file ok #1654
librosa read the file ok #1655
librosa read the file ok #1656
librosa read the file ok #1657
librosa read the file ok #1658
librosa read the file ok #1659
librosa read the file ok #1660
librosa 

In [23]:
'''
glassbreak_aggregator_inverted = dict([[v, k] for k, v in glassbreak_aggregator.items()])
for sl in sorted(glassbreak_aggregator.values(), reverse=True):
    print("Max value for slice #" + str(glassbreak_aggregator_inverted[sl]) + " is " + str(sl))
'''

'\nglassbreak_aggregator_inverted = dict([[v, k] for k, v in glassbreak_aggregator.items()])\nfor sl in sorted(glassbreak_aggregator.values(), reverse=True):\n    print("Max value for slice #" + str(glassbreak_aggregator_inverted[sl]) + " is " + str(sl))\n'

In [24]:
print("number of samples: ")
print(len(samples))
'''
i=744
samp=samples[i]
sr=sample_rates[i]
print(np.max(abs(samp)))
print(labels[i])
ipd.Audio(samp, rate=sr)
'''

2448
0.6194935
1


In [25]:
'''
def log_specgram(audio, sample_rate, window_size=20,
                 step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, times, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, times, np.log(spec.T.astype(np.float32) + eps)
'''

"\ndef log_specgram(audio, sample_rate, window_size=20,\n                 step_size=10, eps=1e-10):\n    nperseg = int(round(window_size * sample_rate / 1e3))\n    noverlap = int(round(step_size * sample_rate / 1e3))\n    freqs, times, spec = signal.spectrogram(audio,\n                                    fs=sample_rate,\n                                    window='hann',\n                                    nperseg=nperseg,\n                                    noverlap=noverlap,\n                                    detrend=False)\n    return freqs, times, np.log(spec.T.astype(np.float32) + eps)\n"

In [26]:
'''
i=15
samp=samples[i]
sr=sample_rates[i]

freqs, times, spectrogram = log_specgram(samp, sr)

fig = plt.figure(figsize=(14, 8))
ax1 = fig.add_subplot(211)
ax1.set_title('Raw wave of ' + file)
ax1.set_ylabel('Amplitude')
ax1.plot(np.linspace(0, 1,samp.size), samp)

ax2 = fig.add_subplot(212)
ax2.imshow(spectrogram.T, aspect='auto', origin='lower', 
           extent=[times.min(), times.max(), freqs.min(), freqs.max()])
ax2.set_yticks(freqs[::16])
ax2.set_xticks(times[::16])
ax2.set_title('Spectrogram of ' + file)
ax2.set_ylabel('Freqs in Hz')
ax2.set_xlabel('Seconds')
'''

"\ni=15\nsamp=samples[i]\nsr=sample_rates[i]\n\nfreqs, times, spectrogram = log_specgram(samp, sr)\n\nfig = plt.figure(figsize=(14, 8))\nax1 = fig.add_subplot(211)\nax1.set_title('Raw wave of ' + file)\nax1.set_ylabel('Amplitude')\nax1.plot(np.linspace(0, 1,samp.size), samp)\n\nax2 = fig.add_subplot(212)\nax2.imshow(spectrogram.T, aspect='auto', origin='lower', \n           extent=[times.min(), times.max(), freqs.min(), freqs.max()])\nax2.set_yticks(freqs[::16])\nax2.set_xticks(times[::16])\nax2.set_title('Spectrogram of ' + file)\nax2.set_ylabel('Freqs in Hz')\nax2.set_xlabel('Seconds')\n"

In [27]:
kf = KFold(n_splits=3, shuffle=True)
samples = np.array(samples)
labels = np.array(labels)
for train_index, test_index in kf.split(samples):
    print("TRAIN:", train_index, "TEST:", test_index)
    train_wav, test_wav = samples[train_index], samples[test_index]
    train_label, test_label = labels[train_index], labels[test_index]

TRAIN: [   0    1    2 ... 2445 2446 2447] TEST: [   8   11   15   18   20   21   22   23   27   33   36   37   39   43
   55   56   58   65   68   69   70   74   77   79   81   88   90   92
   93   94   98   99  103  105  106  107  108  114  115  119  120  121
  128  132  133  137  139  140  147  149  151  153  154  155  156  159
  160  161  162  163  173  180  186  192  194  198  202  203  210  211
  213  215  217  219  227  230  237  240  243  248  250  251  252  256
  263  264  266  267  270  272  274  278  279  280  281  289  290  293
  303  307  309  313  315  316  317  318  324  325  327  331  332  333
  337  338  341  342  347  352  353  359  364  365  368  369  372  380
  381  383  385  387  400  402  403  404  407  408  410  411  412  425
  427  429  435  436  438  439  444  447  448  450  451  455  461  465
  469  473  475  476  480  484  487  492  496  497  500  501  513  515
  518  519  521  524  529  541  543  548  550  551  552  555  561  566
  567  571  574  575  577  5

TRAIN: [   0    1    4 ... 2443 2446 2447] TEST: [   2    3    5    6    7    9   10   14   16   19   29   30   34   38
   40   41   42   45   47   49   50   51   52   63   64   66   67   71
   72   73   75   76   78   80   85   87   89   95   96  100  101  102
  104  110  112  116  117  118  125  127  129  130  131  134  135  141
  143  144  157  158  164  165  166  170  171  172  174  176  179  183
  188  189  190  193  196  197  200  201  206  207  208  216  220  221
  222  223  224  225  229  231  238  239  244  247  249  253  254  257
  258  268  273  282  283  288  292  294  296  297  299  301  308  312
  314  321  329  334  335  336  340  345  349  351  354  356  357  361
  371  378  388  389  392  393  394  395  397  398  399  401  406  413
  415  417  418  419  420  422  423  426  428  431  434  440  441  454
  456  457  460  462  464  467  468  479  481  482  483  485  486  488
  489  498  499  502  503  506  507  508  509  510  511  514  516  517
  522  523  526  530  534  5

In [29]:
# Parameters
lr = 0.001
generations = 20000
num_gens_to_wait = 250
batch_size = 256
drop_out_rate = 0.2
input_shape = (44100,1)

In [30]:
#For Conv1D add Channel
train_wav = np.array(train_wav)
test_wav = np.array(test_wav)
train_wav = train_wav.reshape(-1,44100,1)
test_wav = test_wav.reshape(-1,44100,1)
train_label = keras.utils.to_categorical(train_label, 3)
test_label = keras.utils.to_categorical(test_label, 3)

In [31]:
print(train_wav.shape)

(1632, 44100, 1)


In [14]:
input_tensor = Input(shape=input_shape)

x = layers.Conv1D(8, 11, padding='valid', activation='relu', strides=1)(input_tensor)
x = layers.MaxPooling1D(2)(x)
x = layers.Conv1D(16, 7, padding='valid', activation='relu', strides=1)(x)
x = layers.MaxPooling1D(4)(x)
x = layers.Conv1D(32, 5, padding='valid', activation='relu', strides=1)(x)
x = layers.MaxPooling1D(4)(x)
x = layers.Conv1D(64, 5, padding='valid', activation='relu', strides=1)(x)
x = layers.MaxPooling1D(6)(x)
x = layers.Conv1D(128, 3, padding='valid', activation='relu', strides=1)(x)
x = layers.MaxPooling1D(6)(x)
x = layers.Conv1D(256, 3, padding='valid', activation='relu', strides=1)(x)
x = layers.MaxPooling1D(6)(x)
x = layers.Flatten()(x)
x = layers.Dense(100, activation='relu')(x)
x = layers.Dropout(drop_out_rate)(x)
x = layers.Dense(50, activation='relu')(x)
x = layers.Dropout(drop_out_rate)(x)
x = layers.Dense(20, activation='relu')(x)
output_tensor = layers.Dense(3, activation='softmax')(x)

model = tf.keras.Model(input_tensor, output_tensor)

model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.Adam(lr = lr),
             metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 44100, 1)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 44090, 8)          96        
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 22045, 8)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 22039, 16)         912       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 5509, 16)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 5505, 32)          2592      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1376, 32)          0         
__________

In [ ]:
# add callbacks to stop early if it stops improving
model_filename = 'cnnmodel_w_urban.pkl' 
callbacks = [
    EarlyStopping(monitor='val_acc',
                  patience=10,
                  verbose=1,
                  mode='auto'),
    
    ModelCheckpoint(model_filename, monitor='val_acc',
                    verbose=1,
                    save_best_only=True,
                    mode='auto'),
]

In [16]:
model.fit(train_wav, train_label, 
          validation_data=[test_wav, test_label],
          batch_size=batch_size, 
          epochs=50,
          verbose=1)

Train on 579 samples, validate on 289 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
579/579 [==============================] - 4s 7ms/sample - loss: 1.0894 - acc: 0.4957 - val_loss: 1.0343 - val_acc: 0.6747
Epoch 2/50
579/579 [==============================] - 2s 3ms/sample - loss: 1.0312 - acc: 0.6546 - val_loss: 0.9374 - val_acc: 0.6851
Epoch 3/50
579/579 [==============================] - 2s 3ms/sample - loss: 0.9638 - acc: 0.6736 - val_loss: 0.8990 - val_acc: 0.7336
Epoch 4/50
579/579 [==============================] - 2s 3ms/sample - loss: 0.8959 - acc: 0.7772 - val_loss: 0.8288 - val_acc: 0.7647
Epoch 5/50
579/579 [==============================] - 2s 3ms/sample - loss: 0.8225 - acc: 0.7634 - val_loss: 0.7234 - val_acc: 0.7509
Epoch 6/50
579/579 [==============================] - 2s 3ms/sample - loss: 0.7007 - acc: 0.7720 - val_loss: 0.6365 - val_acc: 0.7647
Epoch 7/50
579/579 [==============================] - 2s 3ms/sample - loss: 0.6037 - acc: 0.7824 - val

In [26]:
Y_test_pred = model.predict(test_wav)
y_predicted_classes_test = Y_test_pred.argmax(axis=-1)
y_actual_classes_test= test_label.argmax(axis=-1)
wrong_examples = np.nonzero(y_predicted_classes_test != y_actual_classes_test)

(289,)


In [18]:
print(wrong_examples)

(array([  1,   5,  19,  41,  50,  52,  55,  57,  58,  64,  65,  67,  75,
        81, 112, 115, 116, 120, 121, 135, 138, 158, 193, 209, 224, 225,
       232, 235, 249, 256, 270, 284, 285, 287]),)


In [ ]:
# save the model
model.save("model_w_urban.h5")